In [1]:
import pandas as pd
from datetime import datetime


def get_direct_link(shared_link):
    return 'https://drive.google.com/uc?export=download&id=' + shared_link.split('/')[-2]

link = "https://drive.google.com/file/d/12oCN9BndrDNLWBLgPMZjbB4kHsqgd9XA/view?usp=drive_link"
direct_link = get_direct_link(link)
wiki_data = pd.read_csv(direct_link, encoding='UTF-8')

link = "https://drive.google.com/file/d/125peVnvL3G7_hv_RGNIT0J416sGm6kMo/view?usp=drive_link"
direct_link = get_direct_link(link)
odds_data = pd.read_csv(direct_link)

wiki_data['Date'] = pd.to_datetime(wiki_data['Date'])
wiki_data['date'] = wiki_data['Date']
wiki_data['pid'] = wiki_data.groupby('wiki_title').ngroup()

duplicates = wiki_data.duplicated(subset=['pid', 'Date'])
duplicates = duplicates[duplicates].index
wiki_data = wiki_data.drop(duplicates)

wiki_data = wiki_data.set_index(['pid', 'Date']).sort_index()

In [2]:
wiki_data.sort_values(by=['pid', 'date'], inplace=True)

wiki_data['wiki_med365'] = wiki_data.groupby('pid')['wiki'].rolling(window=365, min_periods=0).median().reset_index(0, drop=True)
wiki_data['wiki_yesterday'] = wiki_data.groupby('pid')['wiki'].shift()

wiki_data = wiki_data.dropna(subset=['month', 'year', 'day'])

In [3]:
odds_data['parsed_date_1'] = pd.to_datetime(odds_data['Date'], format='%d/%m/%y', errors='coerce')
odds_data['parsed_date_2'] = pd.to_datetime(odds_data['Date'], format='%d/%m/%Y', errors='coerce')
odds_data['date'] = odds_data['parsed_date_1'].combine_first(odds_data['parsed_date_2'])
odds_data.drop(columns=['parsed_date_1', 'parsed_date_2'], axis=1, inplace=True)
odds_data['year'] = odds_data['date'].dt.year
odds_data['month'] = odds_data['date'].dt.month
odds_data['day'] = odds_data['date'].dt.day

wiki_data['date'] = pd.to_datetime(wiki_data['date'], format='%d/%m/%Y')
wiki_data['year'] = wiki_data['date'].dt.year
wiki_data['month'] = wiki_data['date'].dt.month
wiki_data['day'] = wiki_data['date'].dt.day 

In [7]:
wiki_columns = ['date','wiki']+(list(wiki_data.columns[7:20]))

In [8]:
merged_data = pd.DataFrame()
j = -1
names = ''
for i, row in wiki_data.iterrows():
    names = row['original_names']
    home_mask = odds_data['HomeTeam'].apply(lambda x: x in names)
    temp_data = odds_data[home_mask].copy()
    temp_data['original_names'] = names
    temp_data = temp_data.merge(pd.DataFrame([row[wiki_columns]]), on=['date'], how='inner')
    merged_data = pd.concat([merged_data,temp_data], ignore_index=True)
    if (i[0] % 50 ==0) & (i[0] > j):
        print(i[0])
        j = i[0]
        

for column in merged_data.columns:
    if column.startswith('wiki'):
        merged_data = merged_data.rename(columns={column: column + '_h'})
merged_data = merged_data.rename(columns={'original_names': 'original_names_h'})

0
50
100
150


In [9]:
j = -1
names = ''
for i, row in wiki_data.iterrows():
    names = row['original_names']
    away_mask = odds_data['AwayTeam'].apply(lambda x: x in names)
    temp_data = odds_data[away_mask].copy()
    temp_data['original_names'] = names
    temp_data = temp_data.merge(pd.DataFrame([row[wiki_columns]]), on=['date'], how='inner')
    merged_data = pd.concat([merged_data,temp_data], ignore_index=True)
    if (i[0] % 50 ==0) & (i[0] > j):
        print(i[0])
        j = i[0]
    
for column in merged_data.columns:
    if column.startswith('wiki'):
        merged_data = merged_data.rename(columns={column: column + '_a'})
merged_data = merged_data.rename(columns={'original_names': 'original_names_a'})

0
50
100
150


In [11]:
merged_data_a = merged_data[merged_data['wiki_h_a'].isnull()].reset_index()
merged_data_h = merged_data[merged_data['wiki_a'].isnull()]

combined_df = merged_data_h.combine_first(merged_data_a)

In [13]:
combined_df.to_csv('football_merged.csv', index=False)